In [6]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')


sp500 = pd.read_html ('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')
symbols_list = sp500['Symbol'].unique().tolist()
# symbols_list
end_date = '2023-11-19'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)
df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()
df.index.names = ['date','ticker']
#df
df.columns = df.columns.str.lower()
#df
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)
# garman klass volatility
# df
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, Length=20))
df

[**********************85%%***************       ]  429 of 503 completed

C:\Users\netot\AppData\Roaming\Python\Python312\site-packages\yfinance\base.py:279: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  endDt = pd.to_datetime(_datetime.datetime.utcfromtimestamp(end))


[*********************100%%**********************]  503 of 503 completed


adj close       close        high         low        open  \
date       ticker                                                               
2015-11-23 A        37.476860   40.029999   40.070000   39.139999   39.290001   
           AAL      40.423805   42.299999   42.740002   41.810001   42.110001   
           AAPL     26.799692   29.437500   29.932501   29.334999   29.817499   
           ABBV     43.283787   61.150002   61.529999   60.840000   61.310001   
           ABT      39.117039   45.439999   46.099998   45.259998   46.029999   
...                       ...         ...         ...         ...         ...   
2023-11-17 YUM     127.660004  127.660004  128.490005  127.250000  128.419998   
           ZBH     111.669998  111.669998  112.660004  111.330002  112.349998   
           ZBRA    218.020004  218.020004  218.699997  215.270004  218.460007   
           ZION     36.070000   36.070000   36.470001   35.400002   36.250000   
           ZTS     174.800003  174.800003  177.699997  173.699997  177.410004   

                        volume  garman_klass_vol        rsi  
date       ticker                                            
2015-11-23 A         6473400.0         -0.000587        NaN  
           AAL       4166100.0         -0.000403        NaN  
           AAPL    129930000.0         -0.004195        NaN  
           ABBV      6405000.0         -0.046763        NaN  
           ABT       4593800.0         -0.010061        NaN  
...                        ...               ...        ...  
2023-11-17 YUM       1089600.0          0.000033  62.631823  
           ZBH       2991400.0          0.000056  60.522122  
           ZBRA       340600.0          0.000123  54.353061  
           ZION      2136200.0          0.000434  62.590903  
           ZTS       2425100.0          0.000174  59.645233  

[994335 rows x 8 columns]